# Breeze API Demo

## Instrument Update

### Download security master

In [ ]:
pip install wget

In [ ]:
import os
def delete_directory(directory_path):
  try:
    if os.path.exists(directory_path):
      files = os.listdir(directory_path)
      for file in files:
        file_path = os.path.join(directory_path, file)
        if os.path.isfile(file_path):
          os.remove(file_path)
      os.rmdir(directory_path)
      print("All files deleted successfully.")
    else:
      print("Directory not found")

  except OSError:
     print("Error occurred while deleting files.")

In [ ]:
def splitAndTrim_string(textString, delimiter):

    # Split the string based on space delimiter
    list_string = [str.strip() for str in textString.split(delimiter)]

    return list_string

def join_string(list_string,delimiter):

    # Join the string based on '-' delimiter
    textString = delimiter.join(list_string)

    return textString

In [ ]:
def RemoveSpaceFromCSVFileHeader(filename):
  try:
    if os.path.exists(filename):
      csvReadFile=open(filename,"r")
      content=csvReadFile.readlines()
      csvReadFile.close()
      firstLine=content[0]
      delimiter=','
      content[0]=join_string(splitAndTrim_string(firstLine,delimiter),delimiter)
      csvWriteFile=open(filename,"w")
      csvWriteFile.writelines(content)
      print("File '"+filename+"' processed successfully.")
    else:
      print("File '"+filename+"' not found.")
  except OSError:
    print("Error occurred while reading file.")

In [ ]:
instrumentDownloadUrl="https://directlink.icicidirect.com/NewSecurityMaster/SecurityMaster.zip"
instrumentSecurityMasterDir="instruments/SecurityMaster"
BSEScripMastercsv="instruments/SecurityMaster/BSEScripMaster.txt"
CDNSEScripMastercsv="instruments/SecurityMaster/CDNSEScripMaster.txt"
FONSEScripMastercsv="instruments/SecurityMaster/FONSEScripMaster.txt"
NSEScripMastercsv="instruments/SecurityMaster/NSEScripMaster.txt"
instrumentSecurityMasterZip="instruments/SecurityMaster.zip"

In [ ]:
delete_directory(instrumentSecurityMasterDir)
delete_directory("instruments")
os.mkdir("instruments")

In [ ]:
import wget


if os.path.exists(instrumentSecurityMasterZip):
    os.remove(instrumentSecurityMasterZip)

wget.download(instrumentDownloadUrl, instrumentSecurityMasterZip)


### Extract SecurityMaster

In [ ]:

from zipfile import ZipFile
if os.path.exists(instrumentSecurityMasterZip):

    with ZipFile(instrumentSecurityMasterZip, 'r') as zObject:
        zObject.extractall(path=instrumentSecurityMasterDir)

    os.remove(instrumentSecurityMasterZip)
else:
    print("The file does not exist.")

In [ ]:
RemoveSpaceFromCSVFileHeader(BSEScripMastercsv)
RemoveSpaceFromCSVFileHeader(CDNSEScripMastercsv)
RemoveSpaceFromCSVFileHeader(FONSEScripMastercsv)
RemoveSpaceFromCSVFileHeader(NSEScripMastercsv)

### Installing Pandas

In [ ]:
pip install pandas

In [ ]:
import pandas as pd

### Reading BSE Scrip Master

In [ ]:
BSEScripMasterDf=pd.read_csv(BSEScripMastercsv)

In [ ]:
print(BSEScripMasterDf.to_string())

In [ ]:
BSEScripMasterDf[BSEScripMasterDf["ShortName"]=="PERSYS"]

### Reading CDNSE Scrip master

In [ ]:
CDNSEScripMasterDf=pd.read_csv(CDNSEScripMastercsv)

In [ ]:
print(CDNSEScripMasterDf.to_string())

In [ ]:
CDNSEScripMasterDf[CDNSEScripMasterDf["ShortName"]=="USDINR"]

### Reading FONSE Scrip master

In [ ]:
FONSEScripMasterDF=pd.read_csv(FONSEScripMastercsv)

In [ ]:
print(FONSEScripMasterDF.to_string())

In [ ]:
FONSEScripMasterDF[FONSEScripMasterDF["ShortName"]=="CNXBAN"]

### Reading NSE Scrip master

In [ ]:
NSEScripMasterDF=pd.read_csv(NSEScripMastercsv)

In [ ]:
NSEScripMasterDF

In [ ]:
NSEScripMasterDF[NSEScripMasterDF['ShortName']=="PERSYS"]

In [ ]:
NSEScripMasterDF[NSEScripMasterDF['ShortName']=="CNXBAN"]

## Connect to Web socket for Real Time data

In [ ]:
session_token="34916401"
api_key="^42y6975650632#94258t282@r718`77"
api_secret="i0td81HE339202156w96102D98497633"

In [ ]:
pip install breeze-connect

In [ ]:
from breeze_connect import BreezeConnect

# Initialize SDK
breeze = BreezeConnect(api_key=api_key)

# Obtain your session key from https://api.icicidirect.com/apiuser/login?api_key=YOUR_API_KEY
# Incase your api-key has special characters(like +,=,!) then encode the api key before using in the url as shown below.
# import urllib
# print("https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key))

# Generate Session
breeze.generate_session(api_secret=api_secret,
                        session_token=session_token)

# Connect to websocket(it will connect to tick-by-tick data server)
breeze.ws_connect()

# Callback to receive ticks.
def on_ticks(ticks):
    print("Ticks: {}".format(ticks))


breeze.on_ticks=on_ticks


## Get Customer details

In [ ]:
data=breeze.get_customer_details(api_session=session_token)
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

## Get demat holdings

In [ ]:
data=breeze.get_demat_holdings()
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

## Get funds

In [ ]:
data=breeze.get_funds()
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"].to_string())
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

## Get Names

In [ ]:
jsonStockName=breeze.get_names(exchange_code = 'NSE',stock_code = 'CNXBAN')
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonStockName)
  print(successDf.to_string())

## Get quote of Stock

In [ ]:
data=breeze.get_quotes(stock_code="PERSYS", exchange_code="NSE",expiry_date="",product_type="",right="",strike_price="")
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

In [ ]:
data=breeze.get_quotes(stock_code="CNXBAN", exchange_code="NFO",expiry_date="29-FEB-2024",product_type="futures",right="",strike_price="")
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

## Get Options Chain Quotes

In [ ]:
data=breeze.get_option_chain_quotes(stock_code="CNXBAN",exchange_code="NFO",expiry_date="29-FEB-2024",product_type="options",right="call",strike_price="46000")
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

## Get Historical Data

In [ ]:
data=breeze.get_historical_data(interval="1minute",from_date="2023-10-27T09:15:00.000Z",to_date="2024-02-10T15:30:00.000Z",stock_code="CNXBAN",exchange_code="NFO",product_type="options", expiry_date="25-JAN-2024",right="call",strike_price="45000")
jsonData=data["Success"]
statusCode=data["Status"]
if statusCode!=200:
  print ("Error: "+data["Error"])
else:
  successDf=pd.json_normalize(jsonData)
  print(successDf.to_string())

## Subscribe to stock using Token

In [ ]:
# subscribe stocks feeds by stock-token
'''Template for stock_token : X.Y! X : exchange code Y : Market Level data Token : ISEC stock code

Value of X can be : 1 for BSE, 4 for NSE, 13 for NDX, 6 for MCX, 4 for NFO,

Value of Y can be : 1 for Level 1 data, 2 for Level 2 data
'''
NSEStockDf=NSEScripMasterDF[NSEScripMasterDF['ShortName']=="CNXBAN"]
NSEStockToken=NSEStockDf['Token'].array[0]
NSELevel1Token='4.1!'+NSEStockToken
NSELevel2Token='4.2!'+NSEStockToken
breeze.subscribe_feeds(stock_token=NSELevel1Token)
breeze.subscribe_feeds(stock_token=NSELevel2Token)


## Unsubscribe to stock using Token

In [ ]:
# unsubscribe stocks feeds by stock-token
NSEStockDf=NSEScripMasterDF[NSEScripMasterDF['ShortName']=="CNXBAN"]
NSEStockToken=NSEStockDf['Token'].array[0]
NSELevel1Token='4.1!'+NSEStockToken
NSELevel2Token='4.2!'+NSEStockToken
breeze.unsubscribe_feeds(stock_token=NSELevel1Token)
breeze.unsubscribe_feeds(stock_token=NSELevel2Token)

## Subscribe to stock feeds

In [ ]:
# subscribe stocks feeds
breeze.subscribe_feeds(exchange_code="NSE", stock_code="PERSYS", product_type="", expiry_date="", strike_price="", right="", get_exchange_quotes=True, get_market_depth=True)

## Unsubscribe to stock feeds

In [ ]:
# unsubscribe stocks feeds
breeze.unsubscribe_feeds(exchange_code="NSE", stock_code="PERSYS", product_type="", expiry_date="", strike_price="", right="", get_exchange_quotes=True, get_market_depth=True)

## Get Order

### List

In [ ]:
breeze.get_order_list(exchange_code="NSE",
                        from_date="2022-08-01T10:00:00.000Z",
                        to_date="2022-08-19T10:00:00.000Z")

### Cancel Order

In [ ]:
breeze.cancel_order(exchange_code="NSE",
                    order_id="20220819N100000001")

### Modify Order

In [ ]:
breeze.modify_order(order_id="202208191100000001",
                    exchange_code="NFO",
                    order_type="limit",
                    stoploss="0",
                    quantity="250",
                    price="290100",
                    validity="day",
                    disclosed_quantity="0",
                    validity_date="2022-08-22T06:00:00.000Z")

## Get Trade

### List

In [ ]:
breeze.get_trade_list(from_date="2022-08-01T06:00:00.000Z",
                        to_date="2022-08-19T06:00:00.000Z",
                        exchange_code="NSE",
                        product_type="",
                        action="",
                        stock_code="")

### Details

In [ ]:
breeze.get_trade_detail(exchange_code="NSE",
                        order_id="20220819N100000005")

## Place Order

### Buy Nifty Call Option

In [ ]:
stock_code="NIFTY",
exchange_code="NFO",
product="options",
action="buy",
order_type="market",
stoploss="",
quantity="50",
price="",
validity="day",
validity_date="2022-08-30T06:00:00.000Z",
disclosed_quantity="0",
expiry_date="2022-09-29T06:00:00.000Z",
right="call",
strike_price="16600"

In [ ]:
breeze.place_order(stock_code="NIFTY",
                    exchange_code="NFO",
                    product="options",
                    action="buy",
                    order_type="market",
                    stoploss="",
                    quantity="50",
                    price="",
                    validity="day",
                    validity_date="2022-08-30T06:00:00.000Z",
                    disclosed_quantity="0",
                    expiry_date="2022-09-29T06:00:00.000Z",
                    right="call",
                    strike_price="16600")

### SquareOff Nifty call Option

In [ ]:
breeze.square_off(exchange_code="NFO",
                    product="options",
                    stock_code="NIFTY",
                    expiry_date="2022-08-25T06:00:00.000Z",
                    right="Call",
                    strike_price="16600",
                    action="sell",
                    order_type="market",
                    validity="day",
                    stoploss="0",
                    quantity="50",
                    price="0",
                    validity_date="2022-08-12T06:00:00.000Z",
                    trade_password="",
                    disclosed_quantity="0")